In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
from os import listdir
import json

In [6]:
def createNodeLink(dataDict, folder):
    timesteps = list(allNeighbors.keys())
#     print(timesteps)
    cluster = []
    for i in range(1, len(timesteps)):
        childs = timesteps[i-1]
        childKey = list(allNeighbors[childs].keys())
    #     print(childKey)
        parents = timesteps[i]
        parentKey = list(allNeighbors[parents].keys())
    #     print(parentKey)
        for child in childKey:
    #         print(child)
            for parent in parentKey:
    #             print(allNeighbors[timesteps[i-1]][child])
                if set(allNeighbors[timesteps[i-1]][child]).issubset(allNeighbors[timesteps[i]][parent]) :
    #                 print(timesteps[i-1])
                    cluster.append({"source":child, "target":parent})
    #                 cluster['{} - {}'.format(timesteps[i-1], child)] = '{} - {}'.format(timesteps[i], parent)
                    break
                elif (set(allNeighbors[timesteps[i-1]][child]) & set(allNeighbors[timesteps[i]][parent])) :
                    cluster.append({"source":child, "target":parent})

    return cluster
    # cl = cluster.keys()
#     for c in cluster:
#         print(c)
#     print('=' * 100)
    
#     with open('nodeLinkData/output/graphD/{}-cluster.json'.format(folder), 'w', encoding='utf-8') as outfile:
#         json.dump(allNeighbors, outfile, sort_keys=True, indent=4)
    
#     with open('nodeLinkData/output/graphD/{}-network.json'.format(folder), 'w', encoding='utf-8') as outfile:
#         json.dump(cluster, outfile, sort_keys=True, indent=4)

In [12]:
folders = [f for f in listdir("nodeLinkData/output/withOutliers/")]
# print(folders)
for folder in folders:
    onlyfiles = [f for f in listdir("nodeLinkData/output/withOutliers/{}".format(folder)) ]
#     onlyfiles = ["0.06.csv"]
#     print(onlyfiles)
    allNeighbors = {}
#     count = 0
    for file in onlyfiles:
#         print(file)
        path = "nodeLinkData/output/withOutliers/{}/{}".format(folder, file)
        name = file.replace(".csv","")
        contrails = pd.read_csv(path)    
        
        nodeClusters = sorted(contrails['nodeCluster'].value_counts().index.tolist())
#         print(nodeClusters)
        
        contrailsGroup = {}
        for node in nodeClusters:
            ids = contrails[contrails.nodeCluster == node]['origId'].unique().tolist()
            contrailsGroup[node] = ids
            
        allNeighbors[name] = contrailsGroup
#     print(allNeighbors)
    createNodeLink(allNeighbors, folder)


In [67]:
# working with edge crossing
folders = [f for f in listdir("nodeLinkData/output/withOutliers/")]
# folders = ['contrails3']
# print(folders)
for folder in folders:
    onlyfiles = [f for f in listdir("nodeLinkData/output/withOutliers/{}".format(folder)) ]
#     onlyfiles = ["0.06.csv"]
#     print(onlyfiles)
    allNeighbors = {}
#     count = 0
    for file in onlyfiles:
#         print(file)
        path = "nodeLinkData/output/withOutliers/{}/{}".format(folder, file)
        name = file.replace(".csv","")
        contrails = pd.read_csv(path)    
        
        nodeClusters = sorted(contrails['nodeCluster'].value_counts().index.tolist())
#         print(nodeClusters)
        clusterList.append(nodeClusters)
        
        contrailsGroup = {}
        for node in nodeClusters:
            ids = contrails[contrails.nodeCluster == node]['origId'].unique().tolist()
            contrailsGroup[node] = ids
            
        allNeighbors[name] = contrailsGroup
    #     print(allNeighbors)
    
    nDict = []
    keys = allNeighbors.keys()
    # print(keys)
    for key in keys:
        timeKey = allNeighbors[key].keys()
        nDict.append({"time": key, 'cluster': list(timeKey)})
#         print(key, timeKey)
          
#     print(nDict)
    nodelink = createNodeLink(allNeighbors, folder)
#     print(nodelink)
    slist = []
    sDict = {}
    for index, value in enumerate(nodelink):
#         pass
#         print(index, value)
        if value["target"] not in slist:
            slist.append(value['target'])
            sDict[value['target']] =  [value['source']]
        else:
#             pass
            sDict[value['target']].append(value['source'])

            
#     print(nDict)
#     print(sDict)
    print("=" * 100)
#     print(nodelink)
    
    sKeys = list(sDict.keys())
#     print(sKeys)
    sortedList = {}
    
    for index, value in reversed(list(enumerate(nDict))):
#         print(index, value)
#         add the final timestep as the way it is
#         tl = []
        if index == 0:
            break
        else:
            if(len(nDict) - 1  == index):
                sortedList[value['time']] = value['cluster']
            td = nDict[index - 1]
#             print(td)
            sortedList[td['time']] = []
            for i in value['cluster']:
                if i in sKeys:
                    children = sDict[i]
#                     print(children)
                    for child in children:
                        if child not in sortedList[td['time']]:
                            sortedList[td['time']].append(child)
                        elif child in sortedList[td['time']]:
                            sortedList[td['time']].remove(child)
                            sortedList[td['time']].append(child)
            if td['cluster'] != sortedList[td['time']]:
                remainingChildren = list(set(td['cluster']) - set(sortedList[td['time']]))
                sortedList[td['time']].extend(remainingChildren)

    print(sortedList)

{'0.2': [43, 44, 45, 46, 47, 48], '0.19': [39, 40, 41, 42], '0.18': [35, 36, 37, 38], '0.17': [32, 33, 34], '0.16': [27, 28, 29, 30, 31], '0.15': [23, 24, 25, 26], '0.14': [18, 19, 20, 21, 22], '0.13': [15, 16, 17], '0.12': [13, 14], '0.11': [7, 8, 9, 10, 11, 12], '0.1': [1, 2, 3, 4, 5, 6]}
{'0.2': [47, 48, 49, 50], '0.19': [41, 42, 43, 44, 45, 46], '0.18': [35, 36, 37, 38, 40, 39], '0.17': [32, 33, 34], '0.16': [26, 27, 28, 29, 30, 31], '0.15': [24, 25], '0.14': [23, 22], '0.13': [19, 20, 21], '0.12': [14, 15, 16, 17, 18], '0.11': [6, 7, 8, 9, 10, 11, 12, 13], '0.1': [2, 3, 5, 1, 4]}
{'0.2': [41, 42, 43, 44], '0.19': [38, 39, 40], '0.18': [34, 35, 36, 37], '0.17': [31, 32, 33], '0.16': [27, 28, 29, 30], '0.15': [22, 23, 24, 25, 26], '0.14': [19, 20, 21], '0.13': [15, 16, 17, 18], '0.12': [11, 12, 13, 14], '0.11': [7, 8, 9, 10], '0.1': [1, 2, 3, 4, 5, 6]}
{'219': [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82], '218': [70, 63, 68, 66, 62, 64, 65, 67, 69, 71], '217': [57, 50, 51, 52, 53, 5

In [26]:
# validating the data
folder = 'contrails1'
onlyfiles = [f for f in listdir("nodeLinkData/output/withOutliers/{}".format(folder)) ]
#     onlyfiles = ["0.06.csv"]
#     print(onlyfiles)
allNeighbors = {}
#     count = 0
for file in onlyfiles:
#         print(file)
    path = "nodeLinkData/output/withOutliers/{}/{}".format(folder, file)
    name = file.replace(".csv","")
    contrails = pd.read_csv(path)    

    nodeClusters = sorted(contrails['nodeCluster'].value_counts().index.tolist())

    contrailsGroup = {}
    for node in nodeClusters:
        ids = contrails[contrails.nodeCluster == node]['origId'].unique().tolist()
        contrailsGroup[node] = ids

    allNeighbors[name] = contrailsGroup
    
keys = allNeighbors.keys()
# print(keys)
for key in keys:
    timeKey = allNeighbors[key].keys()
    print(key, timeKey)


0.1 dict_keys([1, 2, 3, 4, 5, 6])
0.11 dict_keys([7, 8, 9, 10, 11, 12])
0.12 dict_keys([13, 14])
0.13 dict_keys([15, 16, 17])
0.14 dict_keys([18, 19, 20, 21, 22])
0.15 dict_keys([23, 24, 25, 26])
0.16 dict_keys([27, 28, 29, 30, 31])
0.17 dict_keys([32, 33, 34])
0.18 dict_keys([35, 36, 37, 38])
0.19 dict_keys([39, 40, 41, 42])
0.2 dict_keys([43, 44, 45, 46, 47, 48])


In [19]:
{'source': 1, 'target': 7}
{'source': 1, 'target': 8}
{'source': 2, 'target': 7}
{'source': 2, 'target': 8}
{'source': 3, 'target': 8}
{'source': 4, 'target': 8}
{'source': 5, 'target': 8}
{'source': 6, 'target': 8}
{'source': 7, 'target': 13}
{'source': 7, 'target': 14}
{'source': 8, 'target': 13}
{'source': 8, 'target': 14}
{'source': 9, 'target': 14}
{'source': 10, 'target': 14}
{'source': 11, 'target': 14}
{'source': 12, 'target': 14}
{'source': 13, 'target': 15}
{'source': 13, 'target': 16}
{'source': 14, 'target': 15}
{'source': 14, 'target': 16}
{'source': 15, 'target': 18}
{'source': 16, 'target': 18}
{'source': 16, 'target': 19}
{'source': 17, 'target': 19}
{'source': 18, 'target': 23}
{'source': 18, 'target': 24}
{'source': 19, 'target': 23}
{'source': 19, 'target': 24}
{'source': 20, 'target': 24}
{'source': 21, 'target': 24}
{'source': 22, 'target': 24}
{'source': 23, 'target': 27}
{'source': 24, 'target': 27}
{'source': 24, 'target': 28}
{'source': 25, 'target': 28}
{'source': 26, 'target': 28}
{'source': 27, 'target': 32}
{'source': 27, 'target': 33}
{'source': 28, 'target': 32}
{'source': 28, 'target': 33}
{'source': 29, 'target': 33}
{'source': 30, 'target': 33}
{'source': 31, 'target': 33}
{'source': 32, 'target': 35}
{'source': 33, 'target': 35}
{'source': 33, 'target': 36}
{'source': 34, 'target': 36}
{'source': 35, 'target': 39}
{'source': 35, 'target': 40}
{'source': 36, 'target': 39}
{'source': 36, 'target': 40}
{'source': 37, 'target': 40}
{'source': 38, 'target': 40}
{'source': 39, 'target': 43}
{'source': 40, 'target': 43}
{'source': 40, 'target': 44}
{'source': 41, 'target': 44}
{'source': 42, 'target': 44}

{'source': 1, 'target': 7}
{'source': 1, 'target': 8}
{'source': 2, 'target': 7}
{'source': 2, 'target': 8}
{'source': 3, 'target': 8}
{'source': 4, 'target': 8}
{'source': 5, 'target': 8}
{'source': 6, 'target': 8}
{'source': 7, 'target': 11}
{'source': 8, 'target': 11}
{'source': 8, 'target': 12}
{'source': 9, 'target': 12}
{'source': 10, 'target': 12}
{'source': 11, 'target': 15}
{'source': 12, 'target': 15}
{'source': 12, 'target': 16}
{'source': 13, 'target': 16}
{'source': 14, 'target': 16}
{'source': 15, 'target': 19}
{'source': 15, 'target': 20}
{'source': 16, 'target': 19}
{'source': 16, 'target': 20}
{'source': 17, 'target': 20}
{'source': 18, 'target': 20}
{'source': 19, 'target': 22}
{'source': 20, 'target': 22}
{'source': 20, 'target': 23}
{'source': 22, 'target': 27}
{'source': 22, 'target': 28}
{'source': 23, 'target': 27}
{'source': 23, 'target': 28}
{'source': 24, 'target': 28}
{'source': 25, 'target': 28}
{'source': 26, 'target': 28}
{'source': 27, 'target': 31}
{'sou

[1, 4, 5]